In [ ]:
# imports
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
input_path = Path('/kaggle/input/tabular-playground-series-nov-2021/')
list(input_path.iterdir())

In [ ]:
train_df = pd.read_csv(input_path/'train.csv')
test_df = pd.read_csv(input_path/'test.csv')
submission_df = pd.read_csv(input_path/'sample_submission.csv')

train_df.shape, test_df.shape

In [ ]:
submission_df.head(3)

In [ ]:
train_df.head(3)

## Data sanity check

In [ ]:
# null values
train_df.isnull().sum().sum(), test_df.isnull().sum().sum()

In [ ]:
# duplicates check
len(train_df) - len(train_df.drop(['id', 'target'], axis=1).drop_duplicates())

## EDA

1. It is said that, all the variables are continuous
2. Target variable is binary

In [ ]:
test_id = test_df.loc[:, 'id']
train_target = train_df.loc[:, 'target']
train_df.drop(['id', 'target'], axis=1, inplace=True)

In [ ]:
train_target_counts = train_target.value_counts()
labels = train_target_counts.index
counts = train_target_counts.values

plt.bar(labels, counts)
plt.xticks(labels)
plt.show()

In [ ]:
# Scaling the data
from sklearn.preprocessing import StandardScaler, Normalizer
scaler = StandardScaler()
normalizer = Normalizer()

## Model building

In [ ]:
import catboost
import optuna
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, auc, roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df, train_target, test_size=0.25, stratify=train_target,
                                                    shuffle=True, random_state=13)
X_train.shape, X_test.shape

In [ ]:
def objective(trial):
    """
    Objective function to tune XGBoost classifier
    """
    params = {
        'task_type': 'GPU',
        'iterations': trial.suggest_int('iterations', 50, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
    }
    
    # KFold split
    skf = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)
    cv_scores = []
    
    for train_ix, test_ix in skf.split(X_train, y_train):
        X_train_k, X_test_k = X_train.iloc[train_ix], X_train.iloc[test_ix]
        y_train_k, y_test_k = y_train.iloc[train_ix], y_train.iloc[test_ix]
        
        # Scale data
        # X_train_k = scaler.fit_transform(X_train_k)
        # X_test_k = scaler.transform(X_test_k)
    
        booster = catboost.CatBoostClassifier(**params, loss_function='Logloss', eval_metric='AUC')
        booster.fit(X_train_k, y_train_k, eval_set=[(X_test_k, y_test_k)], verbose=0)
        preds = booster.predict_proba(X_test)
        preds = preds[:, 1]
        cv_scores.append(preds)
    cv_score = np.mean(cv_scores, axis=0)
    return roc_auc_score(y_test, cv_score)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600)

In [ ]:
study.best_trial, study.best_params

In [ ]:
test_df_preds = []
skf = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)
for train_ix, test_ix in skf.split(train_df, train_target):
    X_train, X_test = train_df.iloc[train_ix], train_df.iloc[test_ix]
    y_train, y_test = train_target.iloc[train_ix], train_target.iloc[test_ix]
    # xgb model
    cat_model = catboost.CatBoostClassifier(**study.best_params, task_type='GPU', loss_function='Logloss', eval_metric='AUC')
    cat_model.fit(X_train, y_train, verbose=0, eval_set=[(X_test, y_test)])
    print(cat_model.best_score_)
    test_preds = cat_model.predict_proba(test_df.drop('id', axis=1))
    test_preds = test_preds[:, 1]
    test_df_preds.append(test_preds)

In [ ]:
submission_df.target = np.mean(test_df_preds, axis=0)

In [ ]:
submission_df.to_csv('submission.csv', index=False)